# US Tile Grid Map

This is a starter notebook for creating tile grid maps of US states including DC. A tile grid map is used to display data by states in a visually cleaner way than a traditional to-scale map.

The tile grid used here is based on the [New York Times](https://archive.nytimes.com/www.nytimes.com/interactive/2013/06/26/us/scotus-gay-marriage.html) layout. An overview of existing layouts can be found [here](https://kristw.medium.com/whose-grid-map-is-better-quality-metrics-for-grid-map-layouts-e3d6075d9e80).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.cm as cm
import matplotlib.colors as mcolors

## Empty tile grid map

The dataset used contains the followind columns:
* **code**: The two-letter state code
* **name**: The name of the state
* **row**: Horizontal position within the grid
* **column**: Vertical position within the grid

The grid is 8 rows x 12 columns. Row and column names start at 0. The origin is in the bottom-left corner.

In [ ]:
states = pd.read_csv('../input/us-tile-grid/us_tile_grid.csv')

In [ ]:
fig, axs = plt.subplots(1, figsize=(12, 6), dpi=300)
plt.xlim(0, 12)
plt.ylim(0, 8)
plt.axis('off')
axs.set_aspect('equal')
for _, state in states.iterrows():
    box = patches.Rectangle((state.column,state.row),1,1,linewidth=1,edgecolor='white',facecolor='lightgrey')
    plt.text(state.column + 0.5, state.row + 0.5, state.code, horizontalalignment='center', verticalalignment='center', color='dimgrey')
    axs.add_patch(box)

plt.show()

## Tile grid map with discrete values

Let's demonstrate how a tile grid map works with discrete values. As an example, we take the results of the 2016 presidential elections.

In [ ]:
election = pd.read_csv('../input/2016-american-election-dataset/america.csv');
election['winner']=np.where(pd.isna(election['Trump']), 'Clinton', 'Trump')
election = election[['state_code', 'winner']].merge(states, how='inner', left_on='state_code', right_on='code')

In [ ]:
colors = { 'Trump': '#ff8080', 'Clinton': '#34aae0' }
red = patches.Patch(color=colors['Trump'], label='Trump')
blue = patches.Patch(color=colors['Clinton'], label='Clinton')

fig, axs = plt.subplots(1, figsize=(12, 6), dpi=300)
plt.xlim(0, 12)
plt.ylim(0, 8)
plt.axis('off')
axs.set_aspect('equal')
for _, state in election.iterrows():
    box = patches.Rectangle((state.column,state.row),1,1,linewidth=1,edgecolor='white',facecolor=colors[state.winner])
    plt.text(state.column + 0.5, state.row + 0.5, state.code, horizontalalignment='center', verticalalignment='center', color='white', weight='bold')
    axs.add_patch(box)

plt.title('2016 US presidential election results')
plt.legend(handles=[red, blue], handlelength=0.7)
plt.show()

## Tile grid map with continuous values

A tile grid map can also be used to plot continuous values. As an example, let's plot the infection rates of COVID-19 across US as of late May.

In [ ]:
covid = pd.read_csv('../input/covid19-state-data/COVID19_state.csv')
covid['rate']=covid['Infected']/covid['Population']*100000
covid = covid[['State', 'rate']]
covid.columns = ['state_name', 'rate']
covid = covid.merge(states, how='inner', left_on='state_name', right_on='name')
min_rate = min(covid['rate'])
max_rate = max(covid['rate'])

In [ ]:
fig, axs = plt.subplots(1, figsize=(12, 6), dpi=300)
plt.title('COVID-19 infection rate per 100k on 2020/05/20')
plt.xlim(0, 12)
plt.ylim(0, 8)
plt.axis('off')
axs.set_aspect('equal')

cmap_type = 'coolwarm'
cmap = cm.get_cmap(cmap_type)
norm = mcolors.LogNorm(vmin=min_rate, vmax=max_rate)

for _, state in covid.iterrows():
    col = cmap(norm(state.rate))
    box = patches.Rectangle((state.column,state.row),1,1,linewidth=1,edgecolor='white',facecolor=col)
    plt.text(state.column + 0.5, state.row + 0.5, state.code, horizontalalignment='center', verticalalignment='center', color='white', weight='bold')
    axs.add_patch(box)
    
# create a colorbar
sm = cm.ScalarMappable(cmap=cmap_type, norm=norm)
cb = fig.colorbar(sm, orientation='horizontal', fraction=0.046, pad=0.04)

plt.show()

## Conclusion

With this notebook, you have the tools necessary for easily creating your own tile grid map. Have fun!